In [ ]:
# !pip install sklearn
# import sklearn
import sys
print(sys.path)

In [4]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_absolute_percentage_error as mape, mean_squared_error as mse
import matplotlib.pyplot as plt
from sklearn import linear_model
import control
from scipy import signal as sig
from scipy import optimize as opt


In [5]:
# read in the data
df = pd.read_csv('03_full_dataset_clean.csv')
df.head()

,zone,wban,dt,mw,year,date,hour,month,dow,weekend,...,postcovid,temp,precip,rh,pressure,windspeed,rain,snow,recent,test
0,AE,93730,2018-10-01 01:00:00-04:00,833.284,2018,2018-10-01,1,10,Monday,False,...,False,60.0,0.0,93.0,30.24,0.0,False,False,0,0
1,AE,93730,2018-10-01 02:00:00-04:00,811.558,2018,2018-10-01,2,10,Monday,False,...,False,61.0,0.0,90.0,30.22,3.0,False,False,0,0
2,AE,93730,2018-10-01 03:00:00-04:00,803.722,2018,2018-10-01,3,10,Monday,False,...,False,60.0,0.0,93.0,30.22,0.0,False,False,0,0
3,AE,93730,2018-10-01 04:00:00-04:00,821.569,2018,2018-10-01,4,10,Monday,False,...,False,64.0,0.0,93.0,30.23,5.0,False,False,0,0
4,AE,93730,2018-10-01 05:00:00-04:00,873.834,2018,2018-10-01,5,10,Monday,False,...,False,64.0,0.0,93.0,30.23,5.0,False,False,0,0


In [ ]:
df.groupby('zone').size()
#df.isna().sum()

In [ ]:
#exploratory analysis 

## count how many records are in each zone
df.groupby('zone').size()

## count any missing vals by column
df.isna().sum()
##take aways: rh-324, pressure-4402, windspeed-936

## percent of column missing data
df.isna().sum()/df.count()
## take aways: at most, pressure is missing 0.6% of the data

## check the range of values for columns
## first get only numeric columns
df2 = df[['temp', 'precip', 'pressure','windspeed','rain','snow']]
(df2.max() - df2.min()).idxmax()
#col_ranges = df2.max() - df2.min()
#df.loc[:,col_ranges == col_ranges.max()]

In [ ]:
#explore correlation between data points
matrix = df.corr().round(4)
print(matrix)

In [ ]:
# create visual for corrplot
matrix = df.corr().round(4)
sns.heatmap(matrix, annot=False)
plt.show()

########################################################################
## NEXT STEPS: REMOVE SOME COLUMNS WE DON'T NEED -- YEAR or PRECOVID  ##

In [ ]:
# explore VIF of features
#############################################################
## NEXT STEPS: ADD COLUMNS WE WANT TO X -- RUN BELOW CODE  ##
## https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

# the independent variables set
X = df[['temp', 'precip', 'pressure','windspeed','rain','snow']]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature ## BELOW LINE THROWS ERROR
#vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
  
print(vif_data)

In [6]:
## create lag value
zoneData = df.loc[df.zone=='AE']
zoneData["previous_mw"] = zoneData["mw"].shift(1)
zoneData["previous_temp"] = zoneData["temp"].shift(1)

print(zoneData)

      zone   wban                         dt        mw  year        date  \
0       AE  93730  2018-10-01 01:00:00-04:00   833.284  2018  2018-10-01   
1       AE  93730  2018-10-01 02:00:00-04:00   811.558  2018  2018-10-01   
2       AE  93730  2018-10-01 03:00:00-04:00   803.722  2018  2018-10-01   
3       AE  93730  2018-10-01 04:00:00-04:00   821.569  2018  2018-10-01   
4       AE  93730  2018-10-01 05:00:00-04:00   873.834  2018  2018-10-01   
...    ...    ...                        ...       ...   ...         ...   
34900   AE  93730  2022-09-30 19:00:00-04:00  1103.293  2022  2022-09-30   
34901   AE  93730  2022-09-30 20:00:00-04:00  1067.200  2022  2022-09-30   
34902   AE  93730  2022-09-30 21:00:00-04:00  1019.616  2022  2022-09-30   
34903   AE  93730  2022-09-30 22:00:00-04:00   992.122  2022  2022-09-30   
34904   AE  93730  2022-09-30 23:00:00-04:00   911.198  2022  2022-09-30   

       hour  month     dow  weekend  ...  precip    rh  pressure  windspeed  \
0       

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
## loop through each zone and find summary stats
testMAPE = []
manualMAPE = []
for z in df.zone.unique():
    ## grab training data
    zoneData = df.loc[df.zone==z]
    zoneData = zoneData.dropna()
    zoneData["previous_mw"] = zoneData["mw"].shift(48)
    zoneData["previous_temp"] = zoneData["temp"].shift(24)
    zoneData = zoneData.dropna()
    trainData = zoneData.loc[zoneData.test==0]
    testData = zoneData.loc[zoneData.test==1]
    ##print(testData.head())
    
    ## create model with training data
    x = trainData[['month','temp','precip','rh','pressure','windspeed','rain','snow','previous_temp','previous_mw']]
    y = trainData['mw']
    
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    #print('Coefficients: \n', regr.coef_)
    ## predict using test data
    predOutput = regr.predict(testData[['month','temp','precip','rh','pressure','windspeed','rain','snow','previous_temp','previous_mw']])
    #print("prediction: \n", predOutput)
    #print("true value: \n", testData['mw'])
    ## find mape on test data
    mapeVal = mape(testData['mw'], predOutput)
    testMAPE.append(mapeVal)
    manualCalc = np.mean(np.abs((testData['mw'] - predOutput) / testData['mw'])) * 100
    manualMAPE.append(manualCalc)
res_df = pd.DataFrame({"zone":df.zone.unique(), "mape": testMAPE, "manualMAPE": manualMAPE})

In [21]:
## loop through each zone and find summary stats
modelsList = []
for z in df.zone.unique():
    ## grab training data
    zoneData = df.loc[df.zone==z]
    zoneData = zoneData.dropna()
    zoneData = zoneData.dropna()
    trainData = zoneData.loc[zoneData.test==0]
    testData = zoneData.loc[zoneData.test==1]
    ##print(testData.head())
    
    ## create model with training data
    x = trainData[['month','temp','precip','rh','pressure','windspeed','rain','snow']]
    y = trainData['mw']
    
    regr = linear_model.LinearRegression()
    modelsList.append(regr.fit(x, y))

    ## predict using test data
    predOutput = regr.predict(testData[['month','temp','precip','rh','pressure','windspeed','rain','snow']])


model_df = pd.DataFrame({"zone":df.zone.unique(), "model": modelsList})

In [33]:
#model_df
#testDP = testData[['month','temp','precip','rh','pressure','windspeed','rain','snow']]#.iloc[0]
manualDPData = [[10,55.0,0.0,83.0,29.68,0.0,False,False]]
manualDF = pd.DataFrame(manualDPData, columns=['month','temp','precip','rh','pressure','windspeed','rain','snow'])
#regr.predict(testDP)
#print(manualDF)
predValue = model_df.iloc[0].model.predict(manualDF)
predValue[0]

1135.2116383505384

In [38]:
def prepData(dataFile):
    df = pd.read_csv(dataFile) #'03_full_dataset_clean.csv'
    return df
    #X = df[['temp', 'precip', 'pressure','windspeed','rain','snow']]
    
def buildModels(df):
    modelsList = []
    for z in df.zone.unique():
        ## grab training data
        zoneData = df.loc[df.zone==z]
        zoneData = zoneData.dropna()
        trainData = zoneData.loc[zoneData.test==0]
        testData = zoneData.loc[zoneData.test==1]
        
        ## create model with training data
        x = trainData[['month','temp','precip','rh','pressure','windspeed','rain','snow']]
        y = trainData['mw']
    
        regr = linear_model.LinearRegression()
        modelsList.append(regr.fit(x, y))
    # gather models
    model_df = pd.DataFrame({"zone":df.zone.unique(), "model": modelsList})
    return model_df

def getNewData():
    manualDPData = [[10,55.0,0.0,83.0,29.68,0.0,False,False]]
    manualDF = pd.DataFrame(manualDPData, columns=['month','temp','precip','rh','pressure','windspeed','rain','snow'])
    return manualDF

def predMwUsage(dataPoint,models):
    #manualDPData = [[10,55.0,0.0,83.0,29.68,0.0,False,False]]
    #manualDF = pd.DataFrame(manualDPData, columns=['month','temp','precip','rh','pressure','windspeed','rain','snow'])
    predValue = models.iloc[0].model.predict(dataPoint)
    return predValue[0]

df = prepData('03_full_dataset_clean.csv')
model_df = buildModels(df)
newData = getNewData()

predMwUsage(newData,model_df)


1135.176112763424

In [ ]:
num = np.array([2])
den = np.array([3,1])
H = control.tf(num,den)
print('H(s) =',H)
t, y = control.step_response(H)
plt.plot(t,y)
plt.title('step response')
plt.grid()

In [ ]:
from scipy import signal
signal.TransferFunction(num, den)
signal.TransferFunction(num, den, dt=3600)

In [ ]:
class TF_identificator:
    def __init__(self):
        self.tf = None
        self.inputs = None

    def first_order_mdl(self, t, k, pole):
        self.tf = sig.TransferFunction(k, [pole, 1])
        to, yo, xo = sig.lsim2(self.tf, U=self.inputs, T=t)
        return yo

    def second_order_mdl(self, t, k, wn, delta):
        self.tf = sig.TransferFunction(k*(wn**2), [1, 2*delta*wn, wn**2])
        to, yo, xo = sig.lsim2(self.tf, U=self.inputs, T=t)
        return yo

    def identify_first_order(self, t, u, orig_output, method='lm', p0=[1.0, 1.0]):
        self.inputs = u
        params, params_cov = opt.curve_fit(self.first_order_mdl, t, orig_output,
                                           method=method, maxfev=1000, p0=p0)
        return {'k': params[0], 'tau': params[1]}

    def identify_second_order(self, t, u, orig_output, method='lm', p0=[1.0, 1.0, 0.1]):
        self.inputs = u
        params, params_cov = opt.curve_fit(self.second_order_mdl, t, orig_output,
                                           method=method, maxfev=1000, p0=p0)
        return {'k': params[0], 'wn': params[1], 'zeta': params[2]}

In [ ]:
tf_ident = TF_identificator()
t = [360,365] #max(df.temp) # time
k = [10,100] # gain
pole = 3.6#df.temp
params =  tf_ident.first_order_mdl(t, k, pole)
print("Obtained params:")
print(params)



In [ ]:
max(df.temp)
#range(df.date)
resTF = sig.TransferFunction(max(df.temp), [pole, 1]) #-10 25.555
resTF

In [ ]:
s = control.TransferFunction.s
G = (s + 1)/(s**2 + 2*s + 1)

In [1]:
import sympy as sym
from sympy.abc import s,t,x,y,z
from sympy.integrals import laplace_transform
from sympy.integrals import inverse_laplace_transform

# sympy.laplace_transform(f,t,s)
# Laplace transform (t->s)
U = laplace_transform(5*t, t, s)
print('U')
print(U[0])

U
5/s**2


In [ ]:
# Transfer function
G = (s)/(1-s)**2

# Calculate responses
Y1 = G * U1
Y2 = G * U2
Y3 = G * U3

# Inverse Laplace Transform
u1 = inverse_laplace_transform(U1,s,t)
u2 = inverse_laplace_transform(U2,s,t)
u3 = inverse_laplace_transform(U3,s,t)
y1 = inverse_laplace_transform(Y1,s,t)
y2 = inverse_laplace_transform(Y2,s,t)
y3 = inverse_laplace_transform(Y3,s,t)
print('y1')
print(y1)

# generate data for plot
tm = np.linspace(0,8,100)
us = np.zeros(len(tm))
ys = np.zeros(len(tm))

# substitute numeric values for u and y
for u in [u1,u2,u3]:
    for i in range(len(tm)):
        us[i] += u.subs(t,tm[i])
for y in [y1,y2,y3]:
    for i in range(len(tm)):
        ys[i] += y.subs(t,tm[i])

# plot results
plt.figure()
plt.plot(tm,us,label='u(t)')
plt.plot(tm,ys,label='y(t)')
plt.legend()
plt.xlabel('Time')
plt.show()

In [ ]:
## loop through each zone and find summary stats
testMAPE = []
manualMAPE = []
for z in df.zone.unique():
    ## grab training data
    zoneData = df.loc[df.zone==z]
    zoneData = zoneData.dropna()
    zoneData = zoneData.dropna()
    trainData = zoneData.loc[zoneData.test==0]
    testData = zoneData.loc[zoneData.test==1]
    ##print(testData.head())
    
    ## create model with training data
    x = trainData[['month','temp','precip','rh','pressure','windspeed','rain','snow']]
    y = trainData['mw']
    
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    #print('Coefficients: \n', regr.coef_)
    ## predict using test data
    predOutput = regr.predict(testData[['month','temp','precip','rh','pressure','windspeed','rain','snow']])
    #print("prediction: \n", predOutput)
    #print("true value: \n", testData['mw'])
    ## find mape on test data
    mapeVal = mape(testData['mw'], predOutput)
    testMAPE.append(mapeVal)
    manualCalc = np.mean(np.abs((testData['mw'] - predOutput) / testData['mw'])) * 100
    manualMAPE.append(manualCalc)
res_df = pd.DataFrame({"zone":df.zone.unique(), "mape": testMAPE, "manualMAPE": manualMAPE})